In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import csv as csv
import string
import re

#cleaning artist name 
table = str.maketrans('', '', string.punctuation)

def clean_artist_name(artist):
    return artist.translate(table).lower().replace(' ', '')
def is_correct_artist(artist, artist_genius):
    artist = re.search(artist_genius, artist)
    if artist:
        return True
    return False

    
CLIENT_ACCESS_TOKEN = "IZuuhE89dCkruZSCTCrL-A63IzGUfcg4oshj8fYi6soFOGHqnNNCFWZcbrEClx_t"

BASE_URI = "https://api.genius.com"

def _get(path, params=None, headers=None):

    url = '/'.join([BASE_URI, path])

    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)

    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()
        
           
def lyrics_from_song_path(song_path):
    #song_url = base_url + song_api_path
    page_url = "http://genius.com" + song_path
    page = requests.get(page_url)
    html = BeautifulSoup(page.text, "html.parser")
    #remove script tags that they put in the middle of the lyrics
    [h.extract() for h in html('script')]
    #at least Genius is nice and has a tag called 'lyrics'!
    lyrics = html.find("lyrics").get_text()
    return lyrics

In [175]:
#test artist match function

match = is_correct_artist('the2livecrew', '2livecrew' )
match

#test clean_artist_name
clean_artist_name('Rob $tone')

'robtone'

In [3]:
##Read in CSV file
df = pd.read_csv("../data/final-project-data/bbtop15-lyrics-1596-missing.csv")

artist_list=df['artist'].tolist()
artist_list_clean = [clean_artist_name(artist) for artist in artist_list]

song_list=df['track'].tolist()

print(len(song_list))

1596


In [15]:
##Import rapper data 
## loop throug rappers and an
lyric_list=[]

for i in range(0, len(artist_list_clean)):
    
    song_title = song_list[i]
    artist_name = artist_list_clean[i]
    
    #song to be searched for
    data = {'q': song_title}
    
    #returned json value
    json = _get("search", data)

    song_info = None
    hits = json["response"]["hits"]
    #If we cant find the lyrics here, we should hand pick the lyrics
    #if hits:
     #   first_result = json["response"]["hits"][0]
    
    for hit in hits:
        if clean_artist_name(hit["result"]["primary_artist"]["name"]) == artist_name:
            song_info = hit
            break
    
    lyric = None
    if song_info:
        song_path = song_info["result"]["path"]
        lyric = lyrics_from_song_path(song_path)
    #elif first_result:
     #   song_path =  first_result["result"]["path"]

    #Get lyric by scarping
    
    
    
    if lyric:
        lyric_list.append(lyric)
    else:
        lyric = "NA"
        lyric_list.append(lyric)

KeyboardInterrupt: 

In [31]:
##script for mising artist
##Import rapper data 
## loop throug rappers and an
lyric_list=[]
genius_artist_list=[]

for i in range(0, len(artist_list_clean)):
    #init important vars
    song_title = song_list[i]
    artist_name = artist_list_clean[i]
    
    current_page = 1
    next_page = True
    
    #info to be collected
    song_info = None
    lyric = None
    genius_artist_name = None
    
    while next_page and current_page < 5:
        #song to be searched for
        data = {'q': song_title,'page': current_page}
        
        #returned json value
        json = _get("search", data)
    
        hits = json["response"]["hits"]
        
        if hits:
            #loop through serach results and look for artist name. 
            for hit in hits:
                genius_artist_name = hit["result"]["primary_artist"]["name"]
                clean_genius_artist_name = clean_artist_name(genius_artist_name)
                if is_correct_artist(artist_name, clean_genius_artist_name) or is_correct_artist(clean_genius_artist_name, artist_name):
                    song_info = hit
                    next_page = False
                    break
                    
            current_page +=1
            
        else:
            next_page = False
            
        
        

    #Get lyric by scarping
    if song_info:
        song_path = song_info["result"]["path"]
        lyric = lyrics_from_song_path(song_path)
    
    if lyric:
        lyric_list.append(lyric)
    else:
        lyric = "NaN"
        lyric_list.append(lyric)
        
    if genius_artist_name and lyric !="NaN": 
        genius_artist_list.append(genius_artist_name)
    else:
        genius_artist_name = "NaN"
        genius_artist_list.append(genius_artist_name)

In [34]:
len(lyric_list)

print(genius_artist_list[0:200])

['2 Live Crew', 'Kwame', 'NaN', 'NaN', '3rd Bass', 'Run-D.M.C.', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'Donald-D', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'Queen Latifah', 'NaN', 'NaN', 'NaN', 'NaN', '2 Live Crew', 'NaN', 'NaN', 'Maestro', 'Bobby Jimmy', 'NaN', 'MC Lyte', 'NaN', '2 Live Crew', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'David Bowie', 'Kwame', 'NaN', 'Above the Law', 'NaN', 'NaN', 'NaN', '2 Live Crew', 'NaN', 'NaN', 'NaN', 'King T', 'Kyper', 'NaN', 'NaN', 'Jaz', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'Paris (Rapper)', 'Brand Nubian', 'NaN', 'NaN', 'Poison Clan', 'NaN', 'NaN', 'Monie Love', 'Kwame', 'NaN', 'Bas', 'NaN', 'NaN', '2 Live Crew', 'EPMD', 'Yo-Yo', 'NaN', 'NaN', 'Genius', 'NaN', 'NaN', 'NaN', 'Kool Moe Dee', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN

In [116]:
lyric_list1 = lyric_list

In [35]:
df['lyrics'] = lyric_list
df['genius_artist'] = genius_artist_list

In [38]:
df

,Unnamed: 0,Unnamed: 0.1,artist,track,position,date,lyrics,genius_artist
0,0,0,The 2 Live Crew,Me So Horny,1,1989-11-04,"\n\n[Intro]\n""What'll we get for 10 dollars?""\...",2 Live Crew
1,13,13,Kwame,The Rhythm,14,1989-11-04,\n\nLadies and gentlemen\nThe Kit-Kat Club is ...,Kwame
2,15,15,Redhead Kingpin & The F.B.I.,Do The Right Thing,16,1989-11-04,NaN,NaN
3,16,16,High Performance,Do You Really Wanna Party?,17,1989-11-04,NaN,NaN
4,17,17,3rd Bass,Steppin' Into The A.M.,18,1989-11-04,\n\n*sound of clocks ringing*\n\n[Verse One: M...,3rd Bass
5,18,18,Run-D.M.C.,Pause,19,1989-11-04,\n\nArtist: Run-D.M.C\nAlbum: Back From Hell\...,Run-D.M.C.
6,24,24,Mellow Man Ace,Rhyme Fighter,25,1989-11-04,NaN,NaN
7,25,47,Uzi$ Bros,Nothin' But A Gangster,23,1989-11-11,NaN,NaN
8,26,48,Awesome Dre & The Hard Core Committee,Frankly Speaking,24,1989-11-11,NaN,NaN
9,27,49,Ms. Melodie,"Wake Up, Wake Up!",25,1989-11-11,NaN,NaN


In [37]:
still_missing = df[df['lyrics']=='NaN']

In [39]:
still_missing

,Unnamed: 0,Unnamed: 0.1,artist,track,position,date,lyrics,genius_artist
2,15,15,Redhead Kingpin & The F.B.I.,Do The Right Thing,16,1989-11-04,NaN,NaN
3,16,16,High Performance,Do You Really Wanna Party?,17,1989-11-04,NaN,NaN
6,24,24,Mellow Man Ace,Rhyme Fighter,25,1989-11-04,NaN,NaN
7,25,47,Uzi$ Bros,Nothin' But A Gangster,23,1989-11-11,NaN,NaN
8,26,48,Awesome Dre & The Hard Core Committee,Frankly Speaking,24,1989-11-11,NaN,NaN
9,27,49,Ms. Melodie,"Wake Up, Wake Up!",25,1989-11-11,NaN,NaN
10,28,68,The D.O.C.,The D.O.C. & The Doctor,19,1989-11-18,NaN,NaN
12,30,92,Ice Cream Tee,Let's Work,18,1989-11-25,NaN,NaN
13,31,98,Gucci Crew II,Five Dollar High,24,1989-11-25,NaN,NaN
14,32,114,Arabian Prince,She's Got A Big Posse,15,1989-12-02,NaN,NaN


In [40]:
df.to_csv('../data/final-project-data/bbtop15-lyrics-515-missing.csv')

In [41]:
still_missing.to_csv('../data/final-project-data/bbtop15-lyrics-missing.csv')

In [71]:
print(clean_artist_name("lil'wayne"))

lilwayne
